In [1]:
%matplotlib inline

import librosa
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import torch
import os
import glob
import pickle
import copy
import random
import time
import traceback
from multiprocessing import Process

# Make file index dict

In [7]:
train_list = glob.glob('/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/*')

In [8]:
train_list

['/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_106344.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_151403.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_162270.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_150448.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_157718.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_104892.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_144989.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_147787.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_130559.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_101127.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_176958.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_108219.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train/trn_143663.wav',
 '/Lun0/zhiyong/SdSV_2020_deepmine/tas

In [6]:
OPT_INDEX = '/Lun0/zhiyong/sdsvc_t2_train'
# data_dir = '/Lun0/zhiyong/dataset/vox2/dev/aac'
# data_dir_1 = '/Lun0/zhiyong/dataset/vox1/dev/wav'
# data_dir_2test = '/Lun0/zhiyong/dataset/vox2/test/aac'
data_dir = '/Lun0/zhiyong/SdSV_2020_deepmine/task2_train/wav/train'
train_label_dir = '/Lun0/zhiyong/SdSV_2020_deepmine/task2_enrollment/docs/train_labels.txt'
new_start_id = 7323 #vox with 0-7322


spk2id = os.path.join(OPT_INDEX, 'spk2id')
spk2utt_dict = os.path.join(OPT_INDEX, 'spk2utt_dict')
spk2utt_train_dict = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
# spk2utt_val_dict = os.path.join(OPT_INDEX, 'spk2utt_val_dict')
# val_utt_num = 200

In [17]:
train_labels2utt = {}
with open(train_label_dir, 'r') as f:
    for count, line in enumerate(f):
        if count == 0:
            continue
        line = line[:-1]
        utt, label = line.split(' ')
        if label not in train_labels2utt.keys():
            train_labels2utt[label] = [data_dir+'/'+utt+'.wav']
        else:
            train_labels2utt[label].append(data_dir+'/'+utt+'.wav')

In [18]:
len(train_labels2utt)

588

In [21]:
count = 0
for i in train_labels2utt:
    count += len(train_labels2utt[i])
print(count)

85764


In [22]:
spk2utt = {}
# spk2utt_train = {}
# spk2utt_val = {}

with open(spk2id, 'w') as f:
    pass

# label_dir = glob.glob(data_dir+'/*')
# label = [i.split('/')[-1] for i in label_dir]

# label_dir = glob.glob(data_dir_1+'/*')
# label_1 = [i.split('/')[-1] for i in label_dir]

# label_dir = glob.glob(data_dir_2test+'/*')
# label_2test = [i.split('/')[-1] for i in label_dir]

for count, i in enumerate(train_labels2utt):
    with open(spk2id, 'a') as f:
        f.write(str(count+new_start_id)+','+i+'\n')
    spk2utt[count+new_start_id] = train_labels2utt[i]

# for count_1, i in enumerate(label_1):
#     with open(spk2id, 'a') as f:
#         f.write(str(count_1)+','+i+'\n')
#     spk2utt[count_1] = glob.glob(os.path.join(data_dir_1, i)+'/*/*.wav')

# for count_2test, i in enumerate(label_2test):
#     with open(spk2id, 'a') as f:
#         f.write(str(count+1+count_1+1+count_2test)+','+i+'\n')
#     spk2utt[count+1+count_1+1+count_2test] = glob.glob(os.path.join(data_dir_2test, i)+'/*/*.wav')

In [23]:
len(spk2utt)

588

In [26]:
spk2utt.keys()

dict_keys([7323, 7324, 7325, 7326, 7327, 7328, 7329, 7330, 7331, 7332, 7333, 7334, 7335, 7336, 7337, 7338, 7339, 7340, 7341, 7342, 7343, 7344, 7345, 7346, 7347, 7348, 7349, 7350, 7351, 7352, 7353, 7354, 7355, 7356, 7357, 7358, 7359, 7360, 7361, 7362, 7363, 7364, 7365, 7366, 7367, 7368, 7369, 7370, 7371, 7372, 7373, 7374, 7375, 7376, 7377, 7378, 7379, 7380, 7381, 7382, 7383, 7384, 7385, 7386, 7387, 7388, 7389, 7390, 7391, 7392, 7393, 7394, 7395, 7396, 7397, 7398, 7399, 7400, 7401, 7402, 7403, 7404, 7405, 7406, 7407, 7408, 7409, 7410, 7411, 7412, 7413, 7414, 7415, 7416, 7417, 7418, 7419, 7420, 7421, 7422, 7423, 7424, 7425, 7426, 7427, 7428, 7429, 7430, 7431, 7432, 7433, 7434, 7435, 7436, 7437, 7438, 7439, 7440, 7441, 7442, 7443, 7444, 7445, 7446, 7447, 7448, 7449, 7450, 7451, 7452, 7453, 7454, 7455, 7456, 7457, 7458, 7459, 7460, 7461, 7462, 7463, 7464, 7465, 7466, 7467, 7468, 7469, 7470, 7471, 7472, 7473, 7474, 7475, 7476, 7477, 7478, 7479, 7480, 7481, 7482, 7483, 7484, 7485, 7486, 7487,

In [27]:
count = 0
count_list = []
for i in spk2utt:
    count += len(spk2utt[i])
    count_list.append(len(spk2utt[i]))
print(count)

85764


In [28]:
with open(spk2utt_dict, 'wb') as handle:
    pickle.dump(spk2utt, handle)

with open(spk2utt_dict, 'rb') as handle:
    spk2utt = pickle.load(handle)
spk2utt_train = spk2utt

spk2utt_train = copy.deepcopy(spk2utt)
# for i in range(val_utt_num):
#     spk2utt_val[i] = [spk2utt[i][0]]
#     spk2utt_train[i] = spk2utt[i][1:]

In [29]:
count = 0
count_list = []
for i in spk2utt_train:
    count += len(spk2utt_train[i])
    count_list.append(len(spk2utt[i]))
print(count)

85764


In [32]:
with open(spk2utt_train_dict, 'wb') as handle:
    pickle.dump(spk2utt_train, handle)

In [33]:
spk2utt_train_len = os.path.join(OPT_INDEX, 'spk2utt_train_len')
# spk2utt_val_len = os.path.join(OPT_INDEX, 'spk2utt_val_len')

spk2utt_train_len_dict = {}
# spk2utt_val_len_dict = {}

In [35]:
with open(spk2utt_train_dict, 'rb') as handle:
    spk2utt_train = pickle.load(handle)

for i in spk2utt_train:
    for utt in spk2utt_train[i]:
        spk2utt_train_len_dict[utt] = librosa.core.load(utt, sr=16000)[0].shape[0]

with open(spk2utt_train_len, 'wb') as handle:
    pickle.dump(spk2utt_train_len_dict, handle)

# Iterable Dataset & Make train data

In [4]:
fft = librosa.get_fftlib()
class VoxIterableDataset(object):
    def __init__(self, data_dir_dict, data_len_dict, config):        
        with open(data_dir_dict['spk2utt_train_dict'], 'rb') as handle:
            self.spk2utt_train_dict = pickle.load(handle)
        with open(data_dir_dict['music_dict'], 'rb') as handle:
            self.music_dict = pickle.load(handle)
        with open(data_dir_dict['noise_dict'], 'rb') as handle:
            self.noise_dict = pickle.load(handle)
        with open(data_dir_dict['babble_dict'], 'rb') as handle:
            self.babble_dict = pickle.load(handle)
        with open(data_dir_dict['rir_dict'], 'rb') as handle:
            self.rir_dict = pickle.load(handle)
            
        with open(data_len_dict['spk2utt_train_len'], 'rb') as handle:
            self.spk2utt_train_len = pickle.load(handle)
        with open(data_len_dict['music_len'], 'rb') as handle:
            self.music_len = pickle.load(handle)
        with open(data_len_dict['noise_len'], 'rb') as handle:
            self.noise_len = pickle.load(handle)
        with open(data_len_dict['babble_len'], 'rb') as handle:
            self.babble_len = pickle.load(handle)
        
        
        self.random_spkrs_batchlist = None
        self.ramdom_batch_len = None
        self.random_noise_type = None
        
        
        self.possible_babble_num = [3, 4, 5, 6, 7]
        self.possible_babble_snr = [13, 15, 17, 20]
        self.possible_noise_snr = [0, 5, 10, 15]
        self.possible_music_snr = [5, 8, 10, 15]
        
        self.sr = config['sr']
        self.repeats = config['repeats']
        self.batch_size = config['batch_size']
        self.extended_prefectch = config['extended_prefectch']
        
        self.mfcc_dim = 30
        
        # Auxiliary paras
        self.multi_read_count = 0
        self.preload_mem = False
        
        
        
    def __iter__(self):
        return self
    
    def __next__(self):
        assert len(self.ramdom_batch_len) == len(self.random_spkrs_batchlist)
        try:
            batch_frame_len = self.ramdom_batch_len.pop(0)
            batch_spkrs = self.random_spkrs_batchlist.pop(0)
            batch_noise_type = self.random_noise_type.pop(0)
            batched_feats = np.zeros([self.batch_size, batch_frame_len, self.mfcc_dim])
            batched_labels = np.zeros(self.batch_size)
            
            for batch_index, (spkr, noise_type) in enumerate(zip(batch_spkrs, batch_noise_type)):
                
                concat_wav, VAD_result = self._colleting_and_slicing(spkr, batch_frame_len,\
                hop_len=160, extended_prefectch=self.extended_prefectch)
            
                
                if noise_type == 0:
                    aug_wav = concat_wav
                
                elif noise_type == 1:
                    aug_wav = self._add_rebverb(concat_wav)
                   
                elif noise_type == 2:
                    aug_wav = self._add_noise(concat_wav)
                    
                elif noise_type == 3:
                    aug_wav = self._add_music(concat_wav)
                  
                elif noise_type == 4:
                    aug_wav = self._add_babble(concat_wav)
             
                else:
                    raise NotImplementedError
                    
            
                single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
                dct_type=2, n_fft=512, hop_length=160, \
                win_length=None, window='hann', power=2.0, \
                center=True, pad_mode='reflect', n_mels=30, \
                fmin=20, fmax=7600)
                # Note single_feats needs transpose
                out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
                # Apply VAD
                assert out_feats.shape[0] == VAD_result.shape[0]
                out_feats = out_feats[VAD_result.astype(np.bool)]
                batched_feats[batch_index] = out_feats[:batch_frame_len]
                batched_labels[batch_index] = spkr
                
            return batched_feats, batched_labels
        
        except IndexError:
            raise StopIteration

    def process_one_utt(self, utt_dir):
        try:
            concat_wav, _ = librosa.load(utt_dir, sr=self.sr)
            
            VAD_result = self._VAD_detection(concat_wav)
            
            aug_wav = concat_wav

            single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
            dct_type=2, n_fft=512, hop_length=160, \
            win_length=None, window='hann', power=2.0, \
            center=True, pad_mode='reflect', n_mels=30, \
            fmin=20, fmax=7600)
            # Note single_feats needs transpose
            out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
            # Apply VAD
            assert out_feats.shape[0] == VAD_result.shape[0]
            out_feats = out_feats[VAD_result.astype(np.bool)]
            
            batched_feats = out_feats[None, :, :]
                
            return batched_feats
        
        except Exception:
            traceback.print_exc()

    def process_one_utt_noVAD(self, utt_dir):
        try:
            concat_wav, _ = librosa.load(utt_dir, sr=self.sr)
            
            aug_wav = concat_wav

            single_feats = librosa.feature.mfcc(y=aug_wav, sr=self.sr, n_mfcc=30, \
            dct_type=2, n_fft=512, hop_length=160, \
            win_length=None, window='hann', power=2.0, \
            center=True, pad_mode='reflect', n_mels=30, \
            fmin=20, fmax=7600)
            # Note single_feats needs transpose
            out_feats = self._CMVN(single_feats.T, cmn_window = 300, normalize_variance = False)
            
            batched_feats = out_feats[None, :, :]
                
            return batched_feats
        
        except Exception:
            traceback.print_exc()            
    
    def noise_data_preload(self):
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            _, _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            _, _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            _, _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
    
    def noise_data_preload2mem(self):
        print('preloading to memory')
        
        self.music_preload_dict = {}
        self.noise_preload_dict = {}
        self.babble_preload_dict = {}
        self.preload_mem = True
        print('preloading music_dict')
        for count, i in enumerate(self.music_dict):
            self.music_preload_dict[i], _ = librosa.load(self.music_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading noise_dict')        
        for count, i in enumerate(self.noise_dict):
            self.noise_preload_dict[i], _ = librosa.load(self.noise_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)
        print('preloading babble_dict')        
        for count, i in enumerate(self.babble_dict):
            self.babble_preload_dict[i], _ = librosa.load(self.babble_dict[i], sr=self.sr)
            if (count+1)%100 == 0:
                print(count+1)       
        
        
    def get_random_list(self):
        spkrs_list = self.repeats * list(self.spk2utt_train_dict.keys())
        random.shuffle(spkrs_list)
        len_spkrs_list = len(spkrs_list)
        self.random_spkrs_batchlist = [spkrs_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        self.ramdom_batch_len = [random.randint(400, 400) for i in range(len_spkrs_list // self.batch_size)]
        
        noise_type_list = [i%5 for i in range(len_spkrs_list)]

        random.shuffle(noise_type_list)
        self.random_noise_type = [noise_type_list[i*self.batch_size:i*self.batch_size+self.batch_size]\
        for i in range(len_spkrs_list // self.batch_size)]
        
        assert len(self.random_spkrs_batchlist) == len(self.ramdom_batch_len)\
        == len(self.random_noise_type)
        
    def _colleting_and_slicing(self, spkr, batch_frame_len, hop_len=160, extended_prefectch=2.0):
        
        least_wav_len = (batch_frame_len - 1) * hop_len
        concat_utt = np.zeros(0)
        valid_frames_len = 0
        
        # Use to count multi_read_count
        get_count = 0

        while valid_frames_len < batch_frame_len:
#             concat_utt = np.zeros(0)
            if get_count == 0:
                utt_dir = self._get_random_spk_utt(spkr, self.spk2utt_train_dict)
                utt_len = self.spk2utt_train_len[utt_dir]
#             off = self._get_random_offset(least_wav_len, utt_len) / self.sr
            off = self._get_random_offset(least_wav_len+extended_prefectch*self.sr, utt_len) / self.sr
            dur = least_wav_len / self.sr + extended_prefectch
            
            try:
                utt_part, _ = librosa.load(utt_dir, sr=self.sr, offset=off, duration=dur)
            except Exception:
                get_count += 1
                print(utt_dir, off, dur)
                continue
#                 raise Exception(utt_dir+' '+str(off)+' '+str(dur))       
            
            concat_utt = np.append(concat_utt, utt_part)
            detected_frames = self._VAD_detection(concat_utt)
            valid_frames_len = np.sum(detected_frames)

            get_count += 1

        if get_count > 1:
            self.multi_read_count += 1

        VAD_result = detected_frames
        return concat_utt, VAD_result
    
    def _add_rebverb(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = in_wav
        filter_dir = self._get_random_noise(self.rir_dict)
        filter, _ = librosa.load(filter_dir, sr=self.sr)
        
        signal_length = len(signal)
        filter_length = len(filter)
        output_length = signal_length + filter_length - 1
        output = np.zeros(output_length)

        fft_length = 2**np.ceil(np.log2(4 * filter_length)).astype(np.int)
        block_length = fft_length - filter_length + 1


        filter_padded = np.zeros(fft_length)
        filter_padded[0:filter_length] = filter
        filter_padded = fft.rfft(filter_padded)



        for i in range(signal_length//block_length + 1):
            process_length = min(block_length, signal_length - i * block_length);
            signal_block_padded = np.zeros(fft_length)
            signal_block_padded[0:process_length] = signal[i * block_length : i * block_length + process_length]
            signal_block_padded = fft.rfft(signal_block_padded)

            signal_block_padded = filter_padded * signal_block_padded

            signal_block_padded = fft.irfft(signal_block_padded, n=fft_length)

            if (i*block_length + fft_length) <= output_length:
                output[i*block_length : i*block_length + fft_length] += signal_block_padded
            else:
                output[i*block_length : output_length] += signal_block_padded[:output_length-i*block_length]
        
        # shift with max index of filter
        shift_index = np.argmax(filter)
        
        final_out = output[shift_index:shift_index+signal_length]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_noise(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.noise_dict, return_index=True)
            noise_len = self.noise_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
                
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.noise_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_noise_snr[random.randint(0, len(self.possible_noise_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_music(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        total_noise_len = 0
        signal_off = 0
        while total_noise_len < signal_len:
            
            noise_dir, noise_index = self._get_random_noise(self.music_dict, return_index=True)
            noise_len = self.music_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                total_noise_len += signal_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                total_noise_len += noise_len
                if self.preload_mem:
                    noise = self.music_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_music_snr[random.randint(0, len(self.possible_music_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
            signal_off += len(noise)
        
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_babble(self, in_wav):
        power_before_reverb = in_wav.dot(in_wav) / len(in_wav)
        shift_index = 0
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]
        
        signal_len = len(signal)
        signal_off = 0
        bg_spks_num = self.possible_babble_num[random.randint(0, len(self.possible_babble_num)-1)]    
        for _ in range(bg_spks_num):            
            noise_dir, noise_index = self._get_random_noise(self.babble_dict, return_index=True)
            noise_len = self.babble_len[noise_index]
            if noise_len > signal_len:
                noise_off = self._get_random_offset(signal_len, noise_len)
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index][noise_off:noise_off+signal_len]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr, offset=noise_off/self.sr,\
                    duration=signal_len/self.sr)
            else:
                if self.preload_mem:
                    noise = self.babble_preload_dict[noise_index]
                else:
                    noise, _ = librosa.load(noise_dir, sr=self.sr)
                
            snr_db = self.possible_babble_snr[random.randint(0, len(self.possible_babble_snr)-1)]
        
            signal = self._add_db(signal, noise, signal_off, snr_db, power_before_reverb)
            
        output = signal
        final_out = output[shift_index:shift_index+signal_len]
        power_after_reverb = final_out.dot(final_out) / len(final_out)
        final_out = np.sqrt(power_before_reverb/power_after_reverb) * final_out
        out_wav = final_out
        
        return out_wav
    
    def _add_db(self, in_wav, noise, signal_off, snr_db, power_before_reverb):
        signal = np.zeros(len(in_wav))
        signal[:] = in_wav[:]

        noise_power = noise.dot(noise) / len(noise)
        scale_factor = np.sqrt(10**(-snr_db / 10) * power_before_reverb / noise_power)
        noise = scale_factor * noise

        add_length = min(len(noise), len(signal)-signal_off)
        signal[signal_off:signal_off+add_length] += noise[:add_length]
        out_wav = signal      
        
        return out_wav
    
    def _CMVN(self, in_feat, cmn_window = 300, normalize_variance = False):             
        num_frames = in_feat.shape[0]
        dim = in_feat.shape[1]
        last_window_start = -1
        last_window_end = -1
        cur_sum = np.zeros(dim)
        cur_sumsq = np.zeros(dim)

        out_feat = np.zeros([num_frames, dim])

        for t in range(num_frames):
            window_start = 0
            window_end = 0

            window_start = t - int(cmn_window / 2)
            window_end = window_start + cmn_window

            if (window_start < 0):
                window_end -= window_start
                window_start = 0

            if (window_end > num_frames):
                window_start -= (window_end - num_frames)
                window_end = num_frames
                if (window_start < 0):
                    window_start = 0

            if (last_window_start == -1):
                input_part = in_feat[window_start:window_end]
                cur_sum = np.sum(input_part, axis=0, keepdims=False)
                if normalize_variance:
                    cur_sumsq = np.sum(input_part**2, axis=0, keepdims=False)
            else:
                if (window_start > last_window_start):
                    frame_to_remove = in_feat[last_window_start]
                    cur_sum -= frame_to_remove
                    if normalize_variance:
                        cur_sumsq -= frame_to_remove**2

                if (window_end > last_window_end):
                    frame_to_add = in_feat[last_window_end]
                    cur_sum += frame_to_add
                    if normalize_variance:
                        cur_sumsq += frame_to_add**2

            window_frames = window_end - window_start
            last_window_start = window_start
            last_window_end = window_end

            out_feat[t] = in_feat[t] - (1.0 / window_frames) * cur_sum


            if normalize_variance:
                if (window_frames == 1):
                    out_feat[t] = 0.0
                else:
                    variance = (1.0 / window_frames) * cur_sumsq - (1.0 / window_frames**2) * cur_sum**2
                    variance = np.maximum(1.0e-10, variance)
                    out_feat[t] /= variance**(0.5)
                    
        return out_feat

    def _get_random_noise(self, noise_dict, return_index=False):
        dict_len = len(noise_dict)
        i = random.randint(0, dict_len-1)
        noise_dir = noise_dict[i]
        
        if return_index:
            return noise_dir, i
        else:
            return noise_dir
    
    def _get_random_spk_utt(self, spkr, spk2utt):
        this_utts = spk2utt[spkr]
        this_num_utts = len(this_utts)
        i = random.randint(0, this_num_utts-1)
        utt_dir = this_utts[i]
        return utt_dir

    def _get_random_offset(self, expected_length, utt_len):
        if expected_length > utt_len:
            return 0
        
        free_length = utt_len - expected_length
        offset = random.randint(0, free_length)
        return offset
        
    @property
    def _VAD_config(self):
        vad_energy_threshold = -3.0
        vad_energy_mean_scale = 1.0
        vad_frames_context = 0
        vad_proportion_threshold = 0.12
        
        return vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold
        
        
    def _VAD_detection(self, wav):
        vad_energy_threshold, vad_energy_mean_scale,\
        vad_frames_context, vad_proportion_threshold = self._VAD_config
        
        y_tmp = np.pad(wav, int(512 // 2), mode='reflect')
        y_tmp = librosa.util.frame(y_tmp, frame_length=512, hop_length=160)
        y_log_energy = np.log(np.maximum(np.sum(y_tmp**2, axis=0), 1e-15))

        T = len(y_log_energy)
        output_voiced = np.zeros(T)
        if (T == 0):
            raise Exception("zero wave length")

        energy_threshold = vad_energy_threshold
        if (vad_energy_mean_scale != 0.0):
            assert(vad_energy_mean_scale > 0.0)
            energy_threshold += vad_energy_mean_scale * np.sum(y_log_energy) / T


        assert(vad_frames_context >= 0)
        assert(vad_proportion_threshold > 0.0 and vad_proportion_threshold < 1.0);

        for t in range(T):
            num_count = 0
            den_count = 0
            context = vad_frames_context
            for t2 in range(t - context, t + context+1):
                if (t2 >= 0 and t2 < T):
                    den_count+=1
                    if (y_log_energy[t2] > energy_threshold):
                        num_count+=1

            if (num_count >= den_count * vad_proportion_threshold):
                output_voiced[t] = 1.0
            else:
                output_voiced[t] = 0.0
        
        return output_voiced

In [7]:
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
data_dir_dict = {}

# train
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [8]:
data_len_dict = {}

data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')
data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX, 'spk2utt_train_len')
# data_len_dict['spk2utt_val_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')

In [9]:
with open(data_dir_dict['spk2utt_train_dict'], 'rb') as handle:
    spk2utt_train_dict = pickle.load(handle)

In [10]:
spk2utt_train_dict.keys()

dict_keys([7323, 7324, 7325, 7326, 7327, 7328, 7329, 7330, 7331, 7332, 7333, 7334, 7335, 7336, 7337, 7338, 7339, 7340, 7341, 7342, 7343, 7344, 7345, 7346, 7347, 7348, 7349, 7350, 7351, 7352, 7353, 7354, 7355, 7356, 7357, 7358, 7359, 7360, 7361, 7362, 7363, 7364, 7365, 7366, 7367, 7368, 7369, 7370, 7371, 7372, 7373, 7374, 7375, 7376, 7377, 7378, 7379, 7380, 7381, 7382, 7383, 7384, 7385, 7386, 7387, 7388, 7389, 7390, 7391, 7392, 7393, 7394, 7395, 7396, 7397, 7398, 7399, 7400, 7401, 7402, 7403, 7404, 7405, 7406, 7407, 7408, 7409, 7410, 7411, 7412, 7413, 7414, 7415, 7416, 7417, 7418, 7419, 7420, 7421, 7422, 7423, 7424, 7425, 7426, 7427, 7428, 7429, 7430, 7431, 7432, 7433, 7434, 7435, 7436, 7437, 7438, 7439, 7440, 7441, 7442, 7443, 7444, 7445, 7446, 7447, 7448, 7449, 7450, 7451, 7452, 7453, 7454, 7455, 7456, 7457, 7458, 7459, 7460, 7461, 7462, 7463, 7464, 7465, 7466, 7467, 7468, 7469, 7470, 7471, 7472, 7473, 7474, 7475, 7476, 7477, 7478, 7479, 7480, 7481, 7482, 7483, 7484, 7485, 7486, 7487,

In [11]:
spk2utt_train_dict_small = {}
spk2utt_trial_dict_small = {}
for count, i in enumerate(spk2utt_train_dict):
    if count < 488:
        spk2utt_train_dict_small[1211+count] = spk2utt_train_dict[i]
    else:
        spk2utt_trial_dict_small[1211+count] = spk2utt_train_dict[i]

In [20]:
spk2utt_train_dict_small.keys()

dict_keys([1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1270, 1271, 1272, 1273, 1274, 1275, 1276, 1277, 1278, 1279, 1280, 1281, 1282, 1283, 1284, 1285, 1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320, 1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1330, 1331, 1332, 1333, 1334, 1335, 1336, 1337, 1338, 1339, 1340, 1341, 1342, 1343, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1353, 1354, 1355, 1356, 1357, 1358, 1359, 1360, 1361, 1362, 1363, 1364, 1365, 1366, 1367, 1368, 1369, 1370, 1371, 1372, 1373, 1374, 1375,

In [21]:
with open(os.path.join(OPT_INDEX, 'spk2utt_train_dict_small'), 'wb') as handle:
    pickle.dump(spk2utt_train_dict_small, handle)
with open(os.path.join(OPT_INDEX, 'spk2utt_trial_dict_small'), 'wb') as handle:
    pickle.dump(spk2utt_trial_dict_small, handle)

In [22]:
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict_small')

In [24]:
config = {}

config['sr'] = 16000
config['repeats'] = 100
config['batch_size'] = 1
config['extended_prefectch'] = 1.0

def iter_data_preload(dataset, i):
    dataset.get_random_list()
    try:
        start_time = time.time()
        for count, (data, label) in enumerate(dataset):
            with open('/Lun0/zhiyong/sdsvc_t2_train/train_data_small_sparse/'+str(i)+str('_')+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), label.astype(np.int16)), handle)
#             print(time.time()-start_time)
#             with open('/Lun0/zhiyong/sdsvc_t2_train/tmp_data_errlog_1', 'a') as f:
#                 f.write(str(time.time()-start_time)+'\n')
            start_time = time.time()
            if ((count+1)%10000) == 0:
                print((count+1)//10000)
    except Exception:
        with open('/Lun0/zhiyong/sdsvc_t2_train/tmp_data_errlog_1', 'a') as f:
            traceback.print_exc(file=f)
        raise Exception

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)
# dataset.noise_data_preload2mem()
dataset.noise_data_preload()

processes = [Process(target = iter_data_preload, args = (dataset, i)) for i in range(2)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

preloading music_dict
100
200
300
400
500
600
preloading noise_dict
100
200
300
400
500
600
700
800
900
preloading babble_dict
100
200
300
400
1
1
2
2
3
3
4
4
1543.940950870514


# Make train index

In [25]:
488*200

97600

In [26]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'train_data_small_sparse')
expected_len = 97600
workers = 2
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'train_data_small_sparse.csv')

In [27]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [28]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [29]:
count

97600

# make trial

In [34]:
with open(os.path.join(OPT_INDEX, 'spk2utt_trial_dict_small'), 'rb') as handle:
    spk2utt_trial_dict_small = pickle.load(handle)

In [42]:
count = 0
for i in spk2utt_trial_dict_small:
    count += len(spk2utt_trial_dict_small[i])
print(count)

14490


In [23]:
OPT_INDEX_1 = '/Lun0/zhiyong/dataset'
data_dir_dict = {}

# train
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_train_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX_1, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX_1, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX_1, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX_1, 'rir_dict')

In [24]:
data_len_dict = {}

data_len_dict['music_len'] = os.path.join(OPT_INDEX_1, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX_1, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX_1, 'babble_len')
data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX, 'spk2utt_train_len')
# data_len_dict['spk2utt_val_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')

In [28]:
config = {}

config['sr'] = 16000
config['repeats'] = None
config['batch_size'] = None
config['extended_prefectch'] = None
config['out_dir'] = '/Lun0/zhiyong/sdsvc_t2_train/trial_data_small/'

trial_dict_dir = os.path.join(OPT_INDEX, 'spk2utt_trial_dict_small')

def trial_data_preload(dataset, i, trial_dict_dir):
    with open(trial_dict_dir, 'rb') as handle:
        trial_dict = pickle.load(handle)
        
    start_time = time.time()
    count = 0
    for j in trial_dict:
        for line in trial_dict[j]:
            data = dataset.process_one_utt(line)
            label = line.split('/')[-1][:-4]
            with open(config['out_dir']+str(i)+'_'+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), [label]), handle)
            print(time.time()-start_time)
            start_time = time.time()
            count += 1

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)

processes = [Process(target = trial_data_preload, args = (dataset, i, trial_dict_dir)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

7.970787048339844
0.09568214416503906
0.07162857055664062
0.04517817497253418
0.11772346496582031
0.026267051696777344
0.23095703125
0.39521265029907227
0.30336642265319824
0.1292102336883545
0.3331153392791748
0.19955086708068848
0.10549378395080566
0.07919764518737793
0.10640358924865723
0.3774123191833496
0.19918084144592285
0.2307279109954834
0.19574284553527832
0.3441331386566162
0.16498899459838867
0.14908194541931152
0.19398808479309082
0.24889254570007324
0.10834956169128418
0.0637972354888916
0.0650031566619873
0.16168951988220215
0.08496427536010742
0.07648086547851562
0.09900832176208496
0.02625560760498047
0.08376407623291016
0.1414504051208496
0.057369232177734375
0.040392160415649414
0.1705338954925537
0.7836112976074219
0.2490849494934082
0.08530545234680176
0.21913695335388184
0.5410234928131104
0.2864954471588135
0.11587405204772949
0.08620929718017578
0.26518964767456055
0.032135009765625
0.09271383285522461
0.21153616905212402
0.08006024360656738
0.11550784111022949


0.007974624633789062
0.05493474006652832
0.0890343189239502
0.05025672912597656
0.03162360191345215
0.1323080062866211
0.028889894485473633
0.023050785064697266
0.13179278373718262
0.1176598072052002
0.0513918399810791
0.01606154441833496
0.046758413314819336
0.046460628509521484
0.03598284721374512
0.11268186569213867
0.019454479217529297
0.022535324096679688
0.07387661933898926
0.0386500358581543
0.03748798370361328
0.030466318130493164
0.04042410850524902
0.030204057693481445
0.1830606460571289
0.028326988220214844
0.17831969261169434
0.06718063354492188
0.05417346954345703
0.027420520782470703
0.02719712257385254
0.013175010681152344
0.03625988960266113
0.02642989158630371
0.012691974639892578
0.014596939086914062
0.04148435592651367
0.03883934020996094
0.04359769821166992
0.01464533805847168
0.0478215217590332
0.0482785701751709
0.04378199577331543
0.04619145393371582
0.0114898681640625
0.028397321701049805
0.02158069610595703
0.08585548400878906
0.015172958374023438
0.03850936889

0.04865312576293945
0.02405381202697754
0.0451817512512207
0.026067495346069336
0.0327911376953125
0.04004049301147461
0.04676628112792969
0.025220870971679688
0.03541421890258789
0.030923128128051758
0.03136277198791504
0.019239187240600586
0.014071226119995117
0.021603107452392578
0.017856121063232422
0.04097414016723633
0.0434415340423584
0.051842689514160156
0.023006200790405273
0.041776180267333984
0.028171300888061523
0.022443056106567383
0.027190208435058594
0.07228493690490723
0.030019044876098633
0.01363062858581543
0.03919816017150879
0.025595903396606445
0.04029726982116699
0.03909635543823242
0.021361589431762695
0.02166891098022461
0.03622579574584961
0.029955148696899414
0.020288944244384766
0.02585744857788086
0.04838705062866211
0.05249953269958496
0.02307415008544922
0.03559446334838867
0.033498525619506836
0.01413726806640625
0.018524646759033203
0.018862009048461914
0.023082256317138672
0.02685260772705078
0.028162479400634766
0.023384571075439453
0.03879308700561523

0.035897016525268555
0.025051593780517578
0.027057886123657227
0.039173126220703125
0.04752182960510254
0.04201769828796387
0.040871381759643555
0.05004763603210449
0.018031835556030273
0.020064115524291992
0.06719589233398438
0.0245974063873291
0.008975982666015625
0.04356074333190918
0.037656545639038086
0.02707195281982422
0.04280400276184082
0.0424647331237793
0.029360532760620117
0.04414248466491699
0.04583168029785156
0.06044363975524902
0.05581951141357422
0.04149627685546875
0.013538122177124023
0.04976940155029297
0.021921634674072266
0.03431439399719238
0.06729364395141602
0.08485984802246094
0.04916262626647949
0.08037400245666504
0.012993574142456055
0.027746915817260742
0.06945562362670898
0.035152435302734375
0.1828312873840332
0.01967644691467285
0.04039764404296875
0.032108306884765625
0.024156570434570312
0.03089284896850586
0.026799440383911133
0.03312492370605469
0.021024465560913086
0.06476092338562012
0.009063005447387695
0.021511077880859375
0.00863957405090332
0.

2.1972687244415283
1.3971798419952393
1.313539981842041
1.739635705947876
4.108907699584961
0.467421293258667
0.9667513370513916
0.05855107307434082
0.28678274154663086
1.97163987159729
0.46239781379699707
0.07429027557373047
0.36092090606689453
0.2646481990814209
0.03656268119812012
0.2998044490814209
0.7773001194000244
0.8321676254272461
0.8578882217407227
0.009168386459350586
0.043726444244384766
0.09692120552062988
0.054459333419799805
0.13554048538208008
0.1656644344329834
2.8685834407806396
0.41100049018859863
0.026394128799438477
0.263840913772583
1.157127857208252
0.06248354911804199
1.1873576641082764
0.4274613857269287
0.1032097339630127
0.4692668914794922
0.170518159866333
0.29334211349487305
0.25072288513183594
0.19213414192199707
0.03535056114196777
0.010788440704345703
0.02080082893371582
0.02030348777770996
0.5120110511779785
0.02980780601501465
1.9336097240447998
1.0281181335449219
0.1697683334350586
1.2153441905975342
0.713829755783081
0.3317556381225586
0.223875045776

...

0.037073612213134766
0.013927698135375977
0.016885757446289062
0.017002582550048828
0.02785491943359375
0.03777575492858887
0.023759841918945312
0.01916337013244629
0.03456568717956543
0.013216257095336914
0.02713322639465332
0.02323150634765625
0.02130866050720215
0.01797175407409668
0.012658834457397461
0.018262147903442383
0.026245594024658203
0.013047456741333008
0.03781938552856445
0.011188030242919922
0.017562150955200195
0.02629995346069336
0.029883623123168945
0.019441604614257812
0.023482799530029297
0.02915787696838379
0.018626689910888672
0.02840256690979004
0.0808866024017334
0.017884254455566406
0.039913177490234375
0.03452324867248535
0.05104207992553711
0.0314178466796875
0.024317026138305664
0.11610770225524902
0.0536196231842041
0.016313791275024414
0.055240631103515625
0.012454032897949219
0.048119306564331055
0.05050206184387207
0.055235862731933594
0.04654264450073242
0.01653432846069336
0.029491901397705078
0.013106107711791992
0.025950193405151367
0.03830099105834

0.01605391502380371
0.07575511932373047
0.06116008758544922
0.024808168411254883
0.02361917495727539
0.022581100463867188
0.02937030792236328
0.04920053482055664
0.04553484916687012
0.01332712173461914
0.017982959747314453
0.04599142074584961
0.013501644134521484
0.008922815322875977
0.04625105857849121
0.012485265731811523
0.0448145866394043
0.07291030883789062
0.04283785820007324
0.02655768394470215
0.011575937271118164
0.03184866905212402
0.06090950965881348
0.013205528259277344
0.036824941635131836
0.06027340888977051
0.03470492362976074
0.022298812866210938
0.021738052368164062
0.021153926849365234
0.04631662368774414
0.02683877944946289
0.03580737113952637
0.01482248306274414
0.0281374454498291
0.04406857490539551
0.05017518997192383
0.020368576049804688
0.015338420867919922
0.0386204719543457
0.06636524200439453
0.03815746307373047
0.01965928077697754
0.019710063934326172
0.025119543075561523
0.04252433776855469
0.04407811164855957
0.027927398681640625
0.01834583282470703
0.0470

1.0485117435455322
1.2707335948944092
1.9963328838348389
0.6359927654266357
2.182634115219116
3.523883581161499
2.2113234996795654
0.8926291465759277
0.8202190399169922
1.0509560108184814
1.4194645881652832
1.1529457569122314
0.9809703826904297
0.8643782138824463
1.1796085834503174
0.05280137062072754
2.030210018157959
0.4960193634033203
0.07272005081176758
2.0743699073791504
3.013974666595459
1.2114081382751465
4.370327711105347
1.1037144660949707
1.048722505569458
1.1147353649139404
1.1138031482696533
1.087616205215454
1.0201284885406494
1.0942268371582031
1.106203317642212
1.15787935256958
1.1022720336914062
1.0616469383239746
1.1633899211883545
1.1726670265197754
0.38679957389831543
0.7927103042602539
1.7586607933044434
0.05429840087890625
0.020880699157714844
0.11966466903686523
0.2434554100036621
1.1960184574127197
0.12302899360656738
0.5797855854034424
0.3417956829071045
2.1145527362823486
1.2613787651062012
1.1569430828094482
1.7336621284484863
0.2468867301940918
0.012490272521

0.15289855003356934
0.7253079414367676
0.9346864223480225
0.32865428924560547
0.9996953010559082
0.3559451103210449
1.3267796039581299
0.43857765197753906
0.1033322811126709
1.0116751194000244
0.3666675090789795
0.3511781692504883
1.4935805797576904
0.5397946834564209
0.03809475898742676
2.0265092849731445
0.384537935256958
0.028731822967529297
0.27829432487487793
0.09460020065307617
0.5935139656066895
1.606943130493164
0.10349011421203613
0.5794134140014648
0.2139899730682373
0.19613146781921387
0.058611154556274414
0.2962160110473633
0.7471151351928711
1.7822701930999756
1.9501569271087646
1.5906496047973633
1.9475831985473633
1.3644273281097412
1.643341064453125
1.4346380233764648
0.3211967945098877
0.5898559093475342
0.072235107421875
1.7195866107940674
0.11302065849304199
0.1666567325592041
2.672945499420166
0.16869139671325684
2.0858850479125977
1.9773976802825928
1.7474818229675293
1.1302778720855713
1.5498123168945312
0.23997259140014648
2.1516964435577393
1.1555042266845703
2.

0.12375569343566895
0.38872671127319336
0.1963198184967041
0.9333386421203613
1.2721211910247803
1.0014073848724365
1.2887580394744873
0.05254793167114258
0.41472411155700684
0.19922089576721191
0.03579854965209961
0.24574947357177734
1.6593806743621826
0.38623690605163574
0.22296905517578125
0.3862273693084717
2.4754741191864014
0.0694434642791748
0.8597302436828613
1.1858983039855957
0.11170458793640137
1.7373418807983398
0.526630163192749
1.0522677898406982
0.7010986804962158
0.22445917129516602
0.5351536273956299
0.4286079406738281
0.07278037071228027
0.37714338302612305
0.19305181503295898
2.1805717945098877
0.22693705558776855
1.0201926231384277
1.0115714073181152
0.3616964817047119
0.017688274383544922
1.0513427257537842
1.1298346519470215
1.0794332027435303
0.4036116600036621
0.01569223403930664
0.7697319984436035
1.9962794780731201
0.08423638343811035
0.19418096542358398
2.3613030910491943
0.7790038585662842
2.240300416946411
1.7471272945404053
0.9240307807922363
0.40735745429

# make trial index

In [43]:
# OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'trial_data_small')
expected_len = 14490
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'trial_data_small_csv.csv')

In [44]:
len(glob.glob(train_data_dir+'/*'))

14490

In [45]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [47]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [48]:
count

14490

# make trial list

In [59]:
trial_list_out_dir = os.path.join(OPT_INDEX, 'small_tiral_list')
trial_num_per_utt = 8

In [58]:
with open(os.path.join(OPT_INDEX, 'spk2utt_trial_dict_small'), 'rb') as handle:
    spk2utt_trial_dict_small = pickle.load(handle)

In [119]:
count = 0
with open(trial_list_out_dir, 'w') as of:
    for i in spk2utt_trial_dict_small:
        for line in spk2utt_trial_dict_small[i]:
            for c in range(trial_num_per_utt):
                if (c % 2) == 0:
                    tmp_list = copy.deepcopy(spk2utt_trial_dict_small[i])
                    tmp_list.remove(line)
                    label_t = random.choice(tmp_list)
                    label_t = label_t.split('/')[-1]
                    label_e = line.split('/')[-1]
                    of.write(label_e+' '+label_t+' '+'target'+'\n')
                    count += 1
                else:
                    tmp_list = list(spk2utt_trial_dict_small.keys())
                    tmp_list.remove(i)
                    spk_t = random.choice(tmp_list)
                    label_t = random.choice(spk2utt_trial_dict_small[spk_t])
                    label_t = label_t.split('/')[-1]
                    label_e = line.split('/')[-1]
                    of.write(label_e+' '+label_t+' '+'nontarget'+'\n')                    
                    count += 1
print(count)

115920


# Make trial data for vox2test

In [77]:
OPT_INDEX = '/Lun0/zhiyong/dataset'
data_dir_dict = {}

# val
data_dir_dict['spk2utt_train_dict'] = os.path.join(OPT_INDEX, 'spk2utt_val_dict')
data_dir_dict['music_dict'] = os.path.join(OPT_INDEX, 'music_dict')
data_dir_dict['noise_dict'] = os.path.join(OPT_INDEX, 'noise_dict')
data_dir_dict['babble_dict'] = os.path.join(OPT_INDEX, 'babble_dict')
data_dir_dict['rir_dict'] = os.path.join(OPT_INDEX, 'rir_dict')

In [78]:
data_len_dict = {}

data_len_dict['spk2utt_train_len'] = os.path.join(OPT_INDEX, 'spk2utt_val_len')
data_len_dict['music_len'] = os.path.join(OPT_INDEX, 'music_len')
data_len_dict['noise_len'] = os.path.join(OPT_INDEX, 'noise_len')
data_len_dict['babble_len'] = os.path.join(OPT_INDEX, 'babble_len')

In [132]:
data_dir_2test = '/Lun0/zhiyong/dataset/vox2/test/aac'
spk2utt_trial = {}

label_dir = glob.glob(data_dir_2test+'/*')
label_2test = [i.split('/')[-1] for i in label_dir]

for count, i in enumerate(label_2test):
    spk2utt_trial[count] = glob.glob(os.path.join(data_dir_2test, i)+'/*/*.wav')

In [133]:
len(spk2utt_trial)

118

In [135]:
count = 0
for i in spk2utt_trial:
    count += len(spk2utt_trial[i])

In [136]:
count

36237

In [137]:
spk2utt_trial[0][0]

'/Lun0/zhiyong/dataset/vox2/test/aac/id00562/X7FJ3M3bz3c/00127.wav'

In [149]:
with open(os.path.join(OPT_INDEX, 'spk2utt_trial_aux'), 'wb') as handle:
    pickle.dump(spk2utt_trial, handle)

In [140]:
config = {}

config['sr'] = 16000
config['repeats'] = None
config['batch_size'] = None
config['extended_prefectch'] = None
config['out_dir'] = '/Lun0/zhiyong/dataset/trial_aux_data/'

trial_dict = spk2utt_trial

def trial_data_preload(dataset, i, trial_dict):
#     with open(trial_dict_dir, 'rb') as handle:
#         trial_dict = pickle.load(handle)
        
    start_time = time.time()
    count = 0
    for j in trial_dict:
        for line in trial_dict[j]:
            data = dataset.process_one_utt(line)
            lbpart = line.split('/')[-3:]
            lbpart = lbpart[0]+'-'+lbpart[1]+'-'+lbpart[2]
            label = lbpart[:-4]
            with open(config['out_dir']+str(i)+'_'+str(count), 'wb') as handle:
                pickle.dump((data.astype(np.float16), [label]), handle)
            print(time.time()-start_time)
            start_time = time.time()
            count += 1
    print(count)

dataset = VoxIterableDataset(data_dir_dict, data_len_dict, config)

processes = [Process(target = trial_data_preload, args = (dataset, i, trial_dict)) for i in range(1)]
start_time = time.time()
[p.start() for p in processes]
joined = [p.join() for p in processes]
print(time.time()-start_time)

0.03839588165283203
1.0803940296173096
1.0989949703216553
2.1310536861419678
1.1114509105682373
1.1108920574188232
1.7017908096313477
0.19515776634216309
0.8343348503112793
0.1379408836364746
0.17177796363830566
0.11212897300720215
0.08468937873840332
0.08680558204650879
0.09170889854431152
0.0920259952545166
0.10276079177856445
0.3698267936706543
0.18948030471801758
0.2356884479522705
0.3020508289337158
0.23829126358032227
0.24380016326904297
0.6992557048797607
0.18340611457824707
0.05710101127624512
0.32051777839660645
0.4962160587310791
0.04623103141784668
1.2728407382965088
0.22038578987121582
0.3056926727294922
0.1180868148803711
0.07772517204284668
0.2927999496459961
0.08716654777526855
0.19191789627075195
0.1264503002166748
0.36330413818359375
0.10427212715148926
0.07611393928527832
0.0726780891418457
0.23311161994934082
0.23427438735961914
0.12411141395568848
0.07604289054870605
0.09072422981262207
0.05369067192077637
0.22128915786743164
0.11172151565551758
0.18053817749023438


0.2430431842803955
0.5021414756774902
0.14722871780395508
0.033925533294677734
0.09322428703308105
0.11627006530761719
0.21642518043518066
0.11986994743347168
0.13028454780578613
0.13332438468933105
0.14620614051818848
0.22193050384521484
0.42502856254577637
0.10977387428283691
0.25288891792297363
0.2295398712158203
0.051554203033447266
0.1345984935760498
0.16657233238220215
0.11041927337646484
0.027528047561645508
0.1612839698791504
0.0677180290222168
0.14887642860412598
0.08669805526733398
0.05892038345336914
0.05250811576843262
0.06977319717407227
0.21343636512756348
0.4254493713378906
0.20389056205749512
0.12322402000427246
0.4065079689025879
0.15786170959472656
0.11229801177978516
0.2603309154510498
0.12691473960876465
0.040810585021972656
0.2539389133453369
0.0973961353302002
0.25374937057495117
0.16340160369873047
0.05662894248962402
1.119469165802002
1.1161186695098877
1.0563244819641113
0.055375099182128906
0.1546485424041748
0.15051651000976562
0.014900445938110352
0.14670324

0.5613844394683838
0.24544858932495117
0.08940696716308594
0.48285937309265137
0.13216638565063477
0.26271724700927734
1.0919337272644043
0.23630046844482422
0.1927356719970703
0.08859968185424805
0.37571239471435547
0.15751957893371582
0.25419187545776367
0.14236879348754883
0.18367910385131836
0.11347484588623047
0.3966236114501953
0.023694276809692383
0.16012310981750488
0.03239917755126953
0.0767371654510498
0.11388707160949707
0.01247406005859375
0.11276459693908691
0.12373638153076172
0.14901304244995117
0.026525259017944336
0.15580177307128906
0.11089658737182617
0.13308930397033691
0.03626513481140137
0.0950469970703125
0.053601980209350586
0.08789896965026855
0.03612565994262695
0.10004258155822754
0.059635162353515625
0.07354617118835449
0.10189247131347656
0.20099258422851562
0.16631174087524414
0.10276293754577637
0.09460711479187012
0.015864133834838867
0.01746678352355957
0.05739283561706543
0.0698232650756836
0.050110578536987305
0.08842802047729492
0.08893156051635742
0

0.11999320983886719
0.21862030029296875
0.03254389762878418
0.2317647933959961
0.10932803153991699
0.15908002853393555
0.13405466079711914
0.24046587944030762
0.016869544982910156
0.09301280975341797
0.07646012306213379
0.12770986557006836
0.19635438919067383
0.20559024810791016
0.08252358436584473
0.058035850524902344
0.10783600807189941
0.20461177825927734
0.08429360389709473
0.018860340118408203
0.04560685157775879
0.02500152587890625
0.018995046615600586
0.13834023475646973
0.057083845138549805
0.035834312438964844
0.051592111587524414
0.05830073356628418
0.07040953636169434
0.09768199920654297
0.1732313632965088
0.06565165519714355
0.18457579612731934
0.4767742156982422
0.053356170654296875
0.03331470489501953
0.11698317527770996
0.11109495162963867
0.043814897537231445
0.07333207130432129
0.035698652267456055
0.04188990592956543
0.014394760131835938
0.011674880981445312
0.026773691177368164
0.014703512191772461
0.14265036582946777
0.018650531768798828
0.11543893814086914
0.130295

1.1196670532226562
0.14306402206420898
0.21195054054260254
1.1701633930206299
1.039318323135376
1.0782132148742676
0.9405403137207031
0.045868635177612305
0.022127628326416016
0.1393449306488037
0.1758871078491211
0.013756275177001953
0.09235119819641113
0.12418651580810547
0.014251232147216797
0.13159823417663574
0.08361577987670898
0.07046055793762207
0.03964400291442871
0.012585878372192383
0.01702594757080078
0.025847196578979492
0.023932456970214844
0.01871490478515625
0.012414216995239258
0.08560585975646973
0.026319503784179688
0.017789125442504883
0.01493978500366211
0.016534805297851562
0.030297040939331055
0.03103780746459961
0.03734469413757324
0.017360925674438477
0.03625011444091797
0.11458015441894531
0.11600399017333984
0.10575628280639648
0.10442090034484863
0.08392810821533203
0.020888805389404297
0.10788583755493164
0.028270959854125977
0.03263211250305176
0.16793298721313477
0.06650447845458984
0.0545654296875
0.14861702919006348
0.2796483039855957
0.1766481399536132

...

0.09382343292236328
0.17131590843200684
0.03198361396789551
0.4366774559020996
0.09305524826049805
0.17152714729309082
0.15626120567321777
0.2600398063659668
0.12408041954040527
0.10698604583740234
0.22132372856140137
0.10415840148925781
0.16578316688537598
0.15055227279663086
0.13944172859191895
0.08150291442871094
0.10054421424865723
1.0604493618011475
0.04943108558654785
0.05030941963195801
0.39910888671875
0.26465892791748047
0.1294386386871338
0.15398097038269043
0.07119870185852051
0.09541916847229004
0.03605508804321289
0.08014917373657227
0.16194868087768555
0.14488601684570312
0.20962977409362793
0.1699202060699463
0.11306118965148926
0.10506439208984375
0.11995911598205566
0.24901962280273438
0.10572934150695801
0.2431929111480713
0.15013384819030762
0.06901979446411133
0.16182208061218262
0.14232158660888672
0.21943950653076172
0.1778428554534912
0.0845491886138916
0.0937957763671875
0.0935513973236084
0.13830804824829102
0.11192035675048828
0.19503521919250488
0.12787985801

0.010650634765625
0.007785320281982422
0.008246898651123047
0.016596317291259766
0.024843215942382812
0.007903337478637695
0.010560035705566406
0.00774073600769043
0.011985540390014648
0.009319543838500977
0.011376619338989258
0.01306462287902832
0.03710603713989258
0.007878303527832031
0.018537521362304688
0.016273975372314453
0.011805534362792969
0.00819706916809082
0.02354121208190918
0.010453224182128906
0.010022163391113281
0.008284330368041992
0.012581825256347656
0.0125579833984375
0.010149240493774414
0.007706880569458008
0.007872819900512695
0.010485172271728516
0.00961446762084961
0.008087396621704102
0.009924888610839844
0.008939981460571289
0.008138179779052734
0.009386539459228516
0.010501623153686523
0.007585048675537109
0.0075528621673583984
0.008420467376708984
0.008051395416259766
0.08818626403808594
0.04992032051086426
0.09526681900024414
0.07604813575744629
0.017187833786010742
0.029062986373901367
0.028076648712158203
0.02092123031616211
0.05505061149597168
0.021661

0.02696990966796875
0.02292656898498535
0.02318263053894043
0.033247947692871094
0.04458117485046387
0.04144883155822754
0.020659923553466797
0.014877080917358398
0.03263378143310547
0.023328304290771484
0.024013042449951172
0.016147851943969727
0.01526498794555664
0.015671730041503906
0.02034759521484375
0.03316068649291992
0.020442962646484375
0.03184247016906738
0.016332626342773438
0.01811051368713379
0.016872167587280273
0.018923282623291016
0.01876044273376465
0.022435903549194336
0.024015188217163086
0.024014711380004883
0.05419611930847168
0.017337799072265625
0.01802802085876465
0.02038097381591797
0.02620840072631836
0.03165435791015625
0.04485034942626953
0.015814542770385742
0.010774374008178711
0.018368005752563477
0.014797210693359375
0.02132248878479004
0.013852119445800781
0.017074108123779297
0.015405416488647461
0.00890493392944336
0.014343023300170898
0.01605987548828125
0.01573467254638672
0.00995635986328125
0.014997243881225586
0.019057273864746094
0.0093617439270

0.0626983642578125
0.03207087516784668
0.02170872688293457
0.035034894943237305
0.029663801193237305
0.026559829711914062
0.026823997497558594
0.02741527557373047
0.059215545654296875
0.13275957107543945
0.027238130569458008
0.030005693435668945
0.02379775047302246
0.01578688621520996
0.023794889450073242
0.01922750473022461
0.04250812530517578
0.019967317581176758
0.0576322078704834
0.01976799964904785
0.025104999542236328
0.022789955139160156
0.026633739471435547
0.021625995635986328
0.020427942276000977
0.0365753173828125
0.02691507339477539
0.019470691680908203
0.02932906150817871
0.018884897232055664
0.02443718910217285
0.026482343673706055
0.02966785430908203
0.015248298645019531
0.029031991958618164
0.05304384231567383
0.020668983459472656
0.03278636932373047
0.06106925010681152
0.03638267517089844
0.1686704158782959
0.04129385948181152
0.02398824691772461
0.06027340888977051
0.07504010200500488
0.0143585205078125
0.09569931030273438
0.06111741065979004
0.01976156234741211
0.071

0.08171749114990234
0.025434255599975586
0.012426376342773438
0.0443425178527832
0.018608570098876953
0.015390396118164062
0.033676862716674805
0.014866113662719727
0.02391839027404785
0.0767674446105957
0.03469967842102051
0.04661965370178223
0.016735553741455078
0.04154801368713379
0.04471135139465332
0.06581783294677734
0.0189206600189209
0.030959129333496094
0.1462852954864502
0.057646751403808594
0.0291290283203125
0.019215822219848633
0.04466819763183594
0.019219398498535156
0.019925832748413086
0.03428959846496582
0.020203351974487305
0.02799057960510254
0.02244853973388672
0.06646394729614258
0.02490830421447754
0.02285599708557129
0.025604724884033203
0.03173422813415527
0.0254974365234375
0.052197933197021484
0.02124500274658203
0.017911434173583984
0.02199697494506836
0.03914237022399902
0.019710779190063477
0.019640684127807617
0.02089667320251465
0.039140939712524414
0.019130945205688477
0.016793251037597656
0.021178722381591797
0.023596525192260742
0.023381471633911133
0.

# make trial index for vox test2

In [143]:
OPT_INDEX = '/Lun0/zhiyong/dataset'
train_data_dir = os.path.join(OPT_INDEX, 'trial_aux_data')
expected_len = 36237
workers = 1
single_worker_len = int(expected_len / workers)
output = os.path.join(OPT_INDEX, 'trial_aux_data.csv')

In [144]:
len(glob.glob(train_data_dir+'/*'))

36237

In [145]:
assert expected_len == len(glob.glob(train_data_dir+'/*'))

In [146]:
count = 0
with open(output, 'w') as f:
    for i in range(workers):
        for j in range(single_worker_len):
            path = os.path.join(train_data_dir, str(i)+'_'+str(j))
            assert os.path.isfile(path)
            f.write(path+'\n')
            count += 1

In [147]:
count

36237

# make trial list for vox test2

In [148]:
trial_list_out_dir = os.path.join(OPT_INDEX, 'aux_tiral_list')
trial_num_per_utt = 8

In [150]:
with open(os.path.join(OPT_INDEX, 'spk2utt_trial_aux'), 'rb') as handle:
    spk2utt_trial = pickle.load(handle)

In [153]:
count = 0
with open(trial_list_out_dir, 'w') as of:
    for i in spk2utt_trial:
        for line in spk2utt_trial[i]:
            for c in range(trial_num_per_utt):
                if (c % 2) == 0:
                    tmp_list = copy.deepcopy(spk2utt_trial[i])
                    tmp_list.remove(line)
                    label_t = random.choice(tmp_list)
                    lbpart = label_t.split('/')[-3:]
                    label_t = lbpart[0]+'-'+lbpart[1]+'-'+lbpart[2]
                    
                    lbpart = line.split('/')[-3:]
                    label_e = lbpart[0]+'-'+lbpart[1]+'-'+lbpart[2]                   

                    of.write(label_e+' '+label_t+' '+'target'+'\n')
                    count += 1
                else:
                    tmp_list = list(spk2utt_trial.keys())
                    tmp_list.remove(i)
                    spk_t = random.choice(tmp_list)
                    label_t = random.choice(spk2utt_trial[spk_t])
                    lbpart = label_t.split('/')[-3:]
                    label_t = lbpart[0]+'-'+lbpart[1]+'-'+lbpart[2]
                    
                    lbpart = line.split('/')[-3:]
                    label_e = lbpart[0]+'-'+lbpart[1]+'-'+lbpart[2]
                    
                    of.write(label_e+' '+label_t+' '+'nontarget'+'\n')                    
                    count += 1
print(count)

289896
